In [1]:
print("ok")


ok


In [1]:
import os
from dotenv import load_dotenv

load_dotenv()
OPENAI_API_KEY= os.getenv("OPENAI_API_KEY")

In [2]:
from langchain.document_loaders import PyPDFLoader
file_path= "../data/SDG.pdf"
loader= PyPDFLoader(file_path)
data= loader.load()

In [3]:
pdf_data=""

for page in data:
    pdf_data += page.page_content

In [ ]:
pdf_data

In [5]:
from langchain.text_splitter import TokenTextSplitter

splitter = TokenTextSplitter(
    model_name="gpt-3.5-turbo",
    chunk_size=5000,
    chunk_overlap=50,
)

In [6]:
chunks=splitter.split_text(pdf_data)

In [7]:
len(chunks)

1

In [ ]:
chunks

In [9]:
from langchain.docstore.document import Document
doc_ques_gen = [Document(page_content=t) for t in chunks]
doc=splitter.create_documents(chunks)


In [ ]:
doc_ques_gen

In [15]:
type(doc_ques_gen[0])

langchain_core.documents.base.Document

In [69]:
from langchain.text_splitter import TokenTextSplitter

new_splitter = TokenTextSplitter(
    model_name="gpt-3.5-turbo",
    chunk_size=1000,
    chunk_overlap=10,
)

In [70]:
new_docs=new_splitter.split_documents(doc)

In [ ]:
new_docs

In [7]:
from langchain.chat_models import ChatOpenAI

llm_pipeline = ChatOpenAI(
    temperature=0.9, # high temperature means more creativity
    model = "gpt-3.5-turbo",
)

In [16]:
prompt_template = """
You are an expert at creating questions based on coding materials and documentation.
Your goal is to prepare a coder or programmer for their exam and coding tests.
You do this by asking questions about the text below:

------------
{text}
------------

Create questions that will prepare the coders or programmers for their tests.
Make sure not to lose any important information.

QUESTIONS:
"""

In [26]:
from langchain.prompts import PromptTemplate
prompt_ques = PromptTemplate(template=prompt_template, input_variables=["text"])
#recommended by langchain
prompt = PromptTemplate.from_template(prompt_template)
# prompt.format(text="text")

In [27]:
prompt.save("prompt.yaml")

In [ ]:
prompt_files= PromptTemplate.from_file("prompt.yaml")
prompt_files

In [4]:
refine_template = ("""
You are an expert at creating practice questions based on coding material and documentation.
Your goal is to help a coder or programmer prepare for a coding test.
We have received some practice questions to a certain extent: {existing_answer}.
We have the option to refine the existing questions or add new ones.
(only if necessary) with some more context below.
------------
{text}
------------

Given the new context, refine the original questions in English.
If the context is not helpful, please provide the original questions.
QUESTIONS:
"""
)

In [5]:
refined_prompt_ques = PromptTemplate(template=refine_template, input_variables=["text","existing_answer"])

In [ ]:
from langchain.chains.summarize import load_summarize_chain
question_generator_chain = load_summarize_chain(llm= llm_pipeline, 
                                          chain_type="refine", 
                                          verbose=True, 
                                          question_prompt=prompt_ques, 
                                          refine_prompt= refined_prompt_ques)

In [ ]:
response = question_generator_chain.run(new_docs)
print(response)

In [ ]:
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.vectorstores import FAISS
emdeddings = OpenAIEmbeddings()
vectorstore = FAISS.from_documents(new_docs, embeddings=emdeddings)

In [10]:
llm_ans_pipeline = ChatOpenAI(temperature=0.1, model="gpt-3.5-turbo")

In [ ]:
from langchain.chains import RetrievalQA
answer_gen_chain = RetrievalQA.from_chain_type(llm= llm_ans_pipeline,
                                               chain_type = "stuff",
                                               retriever= vectorstore.as_retriever())

In [ ]:
response

In [ ]:
ques_list = response.split("\n")
ques_list

In [ ]:
# Answer each question and save to a file
for question in ques_list:
    print("Question: ", question)
    answer = answer_gen_chain.run(question)
    print("Answer: ", answer)
    print("--------------------------------------------------\\n\\n")
    # Save answer to file
    with open("answers.txt", "a") as f:
        f.write("Question: " + question + "\\n")
        f.write("Answer: " + answer + "\\n")
        f.write("--------------------------------------------------\\n\\n")
    with open("answers.pdf", "a") as f:
        f.write("Question: " + question + "\\n")
        f.write("Answer: " + answer + "\\n")
        f.write("--------------------------------------------------\\n\\n")